In [1]:
import pandas as pd
import yfinance as yf

In [2]:
ticker = ['SPY'] #Change ticker symbol to review any instrument in yahoo finance 
df =yf.download(ticker, interval = '1D',auto_adjust=True, period="1Y")
#2 min data only available in last 60 days 

# df = df.drop(columns = ['Volume', 'Open', 'High', 'Low', 'Adj Close'])
df.tail(10)

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Volume
Date,,,,,
2022-11-01,390.140015,390.390015,383.290009,384.519989,85407600
2022-11-02,383.899994,388.630005,374.760010,374.869995,126990400
2022-11-03,371.470001,374.200012,368.790009,371.010010,87100100
2022-11-04,377.000000,378.869995,370.000000,376.350006,103449600
2022-11-07,377.709991,380.570007,375.529999,379.950012,68286900
2022-11-08,381.109985,385.119995,377.720001,382.000000,84641100
2022-11-09,379.929993,381.140015,373.609985,374.130005,78495500
2022-11-10,388.049988,395.040009,385.640015,394.690002,141455800
2022-11-11,395.589996,399.350006,393.609985,398.510010,93758600


In [3]:
sr_df = df.copy()
sr_df.to_csv('data.csv')

In [4]:
sr_df.tail(10)

,Open,High,Low,Close,Volume
Date,,,,,
2022-11-01,390.140015,390.390015,383.290009,384.519989,85407600
2022-11-02,383.899994,388.630005,374.760010,374.869995,126990400
2022-11-03,371.470001,374.200012,368.790009,371.010010,87100100
2022-11-04,377.000000,378.869995,370.000000,376.350006,103449600
2022-11-07,377.709991,380.570007,375.529999,379.950012,68286900
2022-11-08,381.109985,385.119995,377.720001,382.000000,84641100
2022-11-09,379.929993,381.140015,373.609985,374.130005,78495500
2022-11-10,388.049988,395.040009,385.640015,394.690002,141455800
2022-11-11,395.589996,399.350006,393.609985,398.510010,93758600


In [5]:
#Check if NA values are in data
sr_df=sr_df[sr_df['Volume']!=0]
sr_df.reset_index(drop=True, inplace=True)
sr_df.isna().sum()
sr_df.tail()

,Open,High,Low,Close,Volume
247,381.109985,385.119995,377.720001,382.000000,84641100
248,379.929993,381.140015,373.609985,374.130005,78495500
249,388.049988,395.040009,385.640015,394.690002,141455800
250,395.589996,399.350006,393.609985,398.510010,93758600
251,396.660004,400.179993,394.829987,395.119995,71903500


In [6]:
def support(df1, l, n1, n2): #n1 n2 before and after candle l
    for i in range(l-n1+1, l+1):
        if(df1.Low[i]>df1.Low[i-1]):
            return 0
    for i in range(l+1,l+n2+1):
        if(df1.Low[i]<df1.Low[i-1]):
            return 0
    return 1

#support(df,46,3,2)

def resistance(df1, l, n1, n2): #n1 n2 before and after candle l
    for i in range(l-n1+1, l+1):
        if(df1.High[i]<df1.High[i-1]):
            return 0
    for i in range(l+1,l+n2+1):
        if(df1.High[i]>df1.High[i-1]):
            return 0
    return 1
#resistance(df, 30, 3, 5)

In [7]:
dfpl = sr_df[0:50]
import plotly.graph_objects as go
from datetime import datetime

fig = go.Figure(data=[go.Candlestick(x=dfpl.index,
                open=dfpl['Open'],
                high=dfpl['High'],
                low=dfpl['Low'],
                close=dfpl['Close'])])

fig.show()

In [8]:
sr = []
n1=5
n2=3
for row in range(n1, (len(sr_df)-n2)): #len(df)-n2
    if support(sr_df, row, n1, n2):
        sr.append((row,sr_df.Low[row],1))
    if resistance(sr_df, row, n1, n2):
        sr.append((row,sr_df.High[row],2))
print(sr)

[(47, 415.9441512869428, 1), (123, 381.9245319583339, 1), (161, 389.0418979580837, 2), (188, 429.963802202433, 2)]


In [9]:
s = 0
e = 200
dfpl = sr_df[s:e]
import plotly.graph_objects as go
from datetime import datetime
import matplotlib.pyplot as plt

fig = go.Figure(data=[go.Candlestick(x=dfpl.index,
                open=dfpl['Open'],
                high=dfpl['High'],
                low=dfpl['Low'],
                close=dfpl['Close'])])

c=0
while (1):
    if(c>len(sr)-1 ):#or sr[c][0]>e
        break
    fig.add_shape(type='line', x0=s, y0=sr[c][1],
                  x1=e,
                  y1=sr[c][1]
                  )#x0=sr[c][0]-5 x1=sr[c][0]+5
    c+=1
fig.show()

In [10]:
plotlist1 = [x[1] for x in sr if x[2]==1]
plotlist2 = [x[1] for x in sr if x[2]==2]
plotlist1.sort()
plotlist2.sort()

for i in range(1,len(plotlist1)):
    if(i>=len(plotlist1)):
        break
    if abs(plotlist1[i]-plotlist1[i-1])<=0.005:
        plotlist1.pop(i)

for i in range(1,len(plotlist2)):
    if(i>=len(plotlist2)):
        break
    if abs(plotlist2[i]-plotlist2[i-1])<=0.005:
        plotlist2.pop(i)
plotlist2
#plt.hist(plotlist, bins=10, alpha=0.5)

[389.0418979580837, 429.963802202433]

In [11]:
s = 0
e = 200
dfpl = sr_df[s:e]
import plotly.graph_objects as go
from datetime import datetime
import matplotlib.pyplot as plt

fig = go.Figure(data=[go.Candlestick(x=dfpl.index,
                open=dfpl['Open'],
                high=dfpl['High'],
                low=dfpl['Low'],
                close=dfpl['Close'])])

c=0
while (1):
    if(c>len(plotlist1)-1 ):#or sr[c][0]>e
        break
    fig.add_shape(type='line', x0=s, y0=plotlist1[c],
                  x1=e,
                  y1=plotlist1[c],
                  line=dict(color="MediumPurple",width=3)
                  )
    c+=1

c=0
while (1):
    if(c>len(plotlist2)-1 ):#or sr[c][0]>e
        break
    fig.add_shape(type='line', x0=s, y0=plotlist2[c],
                  x1=e,
                  y1=plotlist2[c],
                  line=dict(color="RoyalBlue",width=1)
                  )
    c+=1    

fig.show()

In [12]:
ss = []
rr = []
n1=4
n2=3
for row in range(n1, (len(sr_df)-n2)): #len(df)-n2
    if support(sr_df, row, n1, n2):
        ss.append((row,sr_df.Low[row]))
    if resistance(df, row, n1, n2):
        rr.append((row,sr_df.High[row]))

In [13]:
s = 0
e = 200
dfpl = sr_df[s:e]
import plotly.graph_objects as go
from datetime import datetime
import matplotlib.pyplot as plt

fig = go.Figure(data=[go.Candlestick(x=dfpl.index,
                open=dfpl['Open'],
                high=dfpl['High'],
                low=dfpl['Low'],
                close=dfpl['Close'])])

c=0
while (1):
    if(c>len(ss)-1 ):
        break
    fig.add_shape(type='line', x0=ss[c][0], y0=ss[c][1],
                  x1=e,
                  y1=ss[c][1],
                  line=dict(color="MediumPurple",width=3)
                  )
    c+=1

c=0
while (1):
    if(c>len(rr)-1 ):
        break
    fig.add_shape(type='line', x0=rr[c][0], y0=rr[c][1],
                  x1=e,
                  y1=rr[c][1],
                  line=dict(color="RoyalBlue",width=1)
                  )
    c+=1    

    
fig.show()

In [14]:
import MeanRevBacktester as MeanRev

In [15]:
date = df.copy()
date = date.reset_index()
beg = date.iloc[0]["Date"]
end = date.iloc[-1]["Date"]
display(beg)
display(end)

Timestamp('2021-11-15 00:00:00')

Timestamp('2022-11-14 00:00:00')

In [16]:
# Spread /fee 
trading_costs = 0.00007

tester = MeanRev.MeanRevBacktester('Close', 30, 2, beg, end, trading_costs)

In [17]:
optimize = tester.optimize_parameters((25, 100, 1), (1, 5, 1))
num = optimize[0]
num

(74, 2)

In [22]:
data = pd.read_csv("data.csv", index_col="Date")
data.tail()

,Open,High,Low,Close,Volume
Date,,,,,
2022-11-08,381.109985,385.119995,377.720001,382.000000,84641100
2022-11-09,379.929993,381.140015,373.609985,374.130005,78495500
2022-11-10,388.049988,395.040009,385.640015,394.690002,141455800
2022-11-11,395.589996,399.350006,393.609985,398.510010,93758600
2022-11-14,396.660004,400.179993,394.829987,395.119995,71903500


In [23]:
data = data.drop(columns = ['Volume', 'Open', 'High', 'Low'])

In [24]:
data['daily_returns'] = data.Close.pct_change()
data.head()

,Close,daily_returns
Date,,
2021-11-15,460.462250,NaN
2021-11-16,462.284668,0.003958
2021-11-17,461.161682,-0.002429
2021-11-18,462.727997,0.003396
2021-11-19,461.900543,-0.001788


In [25]:
import numpy as np

In [26]:
# Bollinger Bands Trading Strategy 

# Set SMA and standard deviations 
SMA = num[0]
dev = num[1]


# Creating column to hold SMA
data["SMA"] = data["Close"].rolling(SMA).mean()

# Creating the lower band 
data["Lower"] = data["SMA"] - data["Close"].rolling(SMA).std() * dev 

# Creating the upper band
data["Upper"] = data["SMA"] + data["Close"].rolling(SMA).std() * dev 

# Creating a column for distance between closing price and the SMA
data["distance"] = data.Close - data.SMA # helper Column

# Creating a column to add a 1 if oversold to go long 
data["position"] = np.where(data.Close < data.Lower, 1, np.nan) # 1. oversold -> go long

# Adding a -1 to the 'position' row if overbought to go short 
data["position"] = np.where(data.Close > data.Upper, -1, data["position"]) # 2. overbought -> go short

# Adding a 0 to the 'position' column to close position when passes SMA 
data["position"] = np.where(data.distance * data.distance.shift(1) < 0, 0, data["position"])

# If 1-3 isn't met the previous position is held 
data["position"] = data.position.ffill().fillna(0) # where 1-3 isn´t applicable -> hold previous position

# Creating a column for strategy returns
data["strategy_returns"] = data.position.shift(1) * data["daily_returns"]

In [27]:
# data.dropna(inplace=True)

In [28]:
data = data.reset_index()
data.tail()

,Date,Close,daily_returns,SMA,Lower,Upper,distance,position,strategy_returns
247,2022-11-08,382.000000,0.005395,389.823770,348.849504,430.798036,-7.823770,1.0,0.005395
248,2022-11-09,374.130005,-0.020602,389.482292,348.410538,430.554047,-15.352287,1.0,-0.020602
249,2022-11-10,394.690002,0.054954,389.350958,348.410795,430.291121,5.339045,0.0,0.054954
250,2022-11-11,398.510010,0.009679,389.191572,348.491046,429.892099,9.318437,0.0,0.000000
251,2022-11-14,395.119995,-0.008507,389.002795,348.547641,429.457949,6.117200,0.0,-0.000000


In [29]:
s = 0
e = (len(data.index) + 1)
dfpl = sr_df[s:e]
import plotly.graph_objects as go
from datetime import datetime
import matplotlib.pyplot as plt

fig = go.Figure(data=[go.Candlestick(x=dfpl.index,
                open=dfpl['Open'],
                high=dfpl['High'],
                low=dfpl['Low'],
                close=dfpl['Close'])])

fig.add_trace(go.Scatter(
    x=data.index,
    y=data['SMA'],
    mode="lines",
    name="SMA",
    line_color="yellow", 
))

fig.add_trace(go.Scatter(
    x=data.index,
    y=data['Lower'],
    mode="lines",
    name="Lower Bound",
    line_color="Black", 
    
))

fig.add_trace(go.Scatter(
    x=data.index,
    y=data['Upper'],
    mode="lines",
    name="Upper Bound",
    line_color="Purple"
))

c=0
while (1):
    if(c>len(ss)-1 ):
        break
    fig.add_shape(type='line', x0=ss[c][0], y0=ss[c][1],
                  x1=e,
                  y1=ss[c][1],
                  line=dict(color="gray",width=1)
                  )
    c+=1

c=0
while (1):
    if(c>len(rr)-1 ):
        break
    fig.add_shape(type='line', x0=rr[c][0], y0=rr[c][1],
                  x1=e,
                  y1=rr[c][1],
                  line=dict(color="violet",width=1)
                  )
    c+=1    

# fig.add_shape(go.Scatter(
#     x=(data[data["position"]==1].index),
#     y=data['Close'],
#     text="dot",
# ))
    
fig.show()